# Benchmarking simple CDR3 Beta Levenshtein distance

## Setup

In [6]:
import os
from pathlib import Path
import sys

if '__project_dir__' not in globals():
    __project_dir__ = Path.cwd().parent.resolve()

sys.path.append(__project_dir__)
os.chdir(__project_dir__)

In [21]:
import numpy as np
import pandas as pd
from pathlib import Path
from tcrdist.repertoire import TCRrep
from tcrdist.setup_tests import download_and_extract_zip_file

### Load data

In [8]:
download_and_extract_zip_file('dash.zip', source = "dropbox", dest ="/home/yutanagano/UCLOneDrive/MBPhD/projects/tcr_embedder/data/tcrdist/")

In [25]:
df = pd.read_csv("/home/yutanagano/UCLOneDrive/MBPhD/projects/tcr_embedder/data/tcrdist/dash_human.csv").dropna()
tr = TCRrep(cell_df = df, 
            organism = 'human', 
            chains = ['beta'])

In [26]:
df.head()

,subject,epitope,count,v_a_gene,j_a_gene,cdr3_a_aa,v_b_gene,j_b_gene,cdr3_b_aa,clone_id
0,human_subject0005,BMLF,1,TRAV5*01,TRAJ31*01,CAADSNARFMF,TRBV20-1*01,TRBJ1-3*01,CSARDRVGNTIYF,clone_0
1,human_subject0005,BMLF,1,TRAV5*01,TRAJ31*01,CAADSNARLMF,TRBV20-1*01,TRBJ1-3*01,CSARDRVGNTIYF,clone_1
2,human_subject0028,M1,1,TRAV13-1*01,TRAJ43*01,CAAENNNDMRF,TRBV7-6*01,TRBJ2-7*01,CASSLGTSYEQYF,clone_2
3,human_subject0029,M1,1,TRAV41*01,TRAJ40*01,CAAETTSGTYKYIF,TRBV14*01,TRBJ2-7*01,CASSPIAGSSYEQYF,clone_3
4,human_subject0011,M1,1,TRAV27*01,TRAJ42*01,CAAGGSQGNLIF,TRBV19*02,TRBJ2-7*01,CASSIRSSYEQYF,clone_4


In [27]:
len(df)

408

In [29]:
tr.pw_beta.shape

(403, 403)

In [22]:
epitope = df['epitope'].to_numpy()
within = (epitope[:, np.newaxis] == epitope[np.newaxis, :]) & (~(np.eye(len(epitope))==1))

In [24]:
within.shape

(408, 408)